<a href="https://colab.research.google.com/github/cwjng/si670-final-project/blob/main/si670_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
# Git repo to clone
# git clone https://github.com/cwjng/si670-final-project.git

In [131]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, f1_score

from imblearn.over_sampling import SMOTE

In [132]:
cmr_df = pd.read_csv("si670-final-project/cancer_mortality_rates.csv")
cmr_df.head()

,cmRate,population,medianIncome,pctPoverty,avgHouse,medianAge,medianAgeM,medianAgeF,pctHS,pctBach,...,pctBlack,pctAsian,pctOther,rmRace,pctMarried,pctMarriedHouse,birthRate,region,state,county
0,191.2,25859,38013,18.6,2.28,45.4,44.6,46.2,36.4,7.9,...,4.925408,0.338357,0.065365,white,62.1,57.173258,4.687790,Southeast,Alabama,Cherokee County
1,187.9,82005,39922,17.2,2.55,40.5,39.4,41.9,34.7,9.1,...,1.190638,0.557031,0.737356,white,57.6,54.850257,3.913309,Southeast,Alabama,Cullman County
2,217.4,41131,26602,35.2,2.53,38.7,36.2,40.0,34.6,8.2,...,69.184419,0.341605,0.161313,black,33.4,30.987632,8.433257,Southeast,Alabama,Dallas County
3,193.8,81468,54298,14.4,2.65,37.9,36.7,39.1,35.4,13.4,...,21.218875,0.516326,0.678529,white,51.0,54.326202,5.800147,Southeast,Alabama,Elmore County
4,205.8,16759,35664,20.6,2.36,43.2,41.3,46.1,38.8,8.4,...,12.387547,0.390625,0.000000,white,53.6,51.499218,5.730897,Southeast,Alabama,Fayette County


In [133]:
corr = cmr_df["pctMarried"].corr(cmr_df["pctMarriedHouse"])
print("Correlation:", corr)

Correlation: 0.8487861233872582


In [105]:
# cmr_df.info()

In [134]:
from sklearn.cluster import KMeans
# import numpy as np

# # Step 1: Run K-means on cmRate only
kmeans = KMeans(n_clusters=3, random_state=42)
cmr_df["cluster_raw"] = kmeans.fit_predict(cmr_df[["cmRate"]])

# Step 2: Sort clusters by actual mean cmRate
# (so cluster 0 = low, 1 = medium, 2 = high)
cluster_order = (
    cmr_df.groupby("cluster_raw")["cmRate"]
    .mean()
    .sort_values()
    .index
)

# Mapping raw clusters → ordered levels (0=low, 1=med, 2=high)
cluster_map = {old: new for new, old in enumerate(cluster_order)}
cmr_df["cmClass"] = cmr_df["cluster_raw"].map(cluster_map)

# Step 3: Drop the raw cluster
cmr_df = cmr_df.drop(columns=["cluster_raw"])

# Step 4: Check distribution
print(cmr_df["cmClass"].value_counts())

# Step 5: Extract labels per county
county_labels = cmr_df[["county", "state", "cmRate", "cmClass"]]
county_labels.head()


cmClass
1    263
0    142
2    132
Name: count, dtype: int64


,county,state,cmRate,cmClass
0,Cherokee County,Alabama,191.2,1
1,Cullman County,Alabama,187.9,1
2,Dallas County,Alabama,217.4,2
3,Elmore County,Alabama,193.8,1
4,Fayette County,Alabama,205.8,2


In [107]:
def categorize_cmrates(rate, q1, median, q3):
  if rate < q1:
    return "Low"
  elif rate < median:
    return "Medium"
  return "High"

quartile_1 = cmr_df["cmRate"].quantile(0.25)
median = cmr_df["cmRate"].quantile(0.5)
quartile_3 = cmr_df["cmRate"].quantile(0.75)

cmr_df["cmClass"] = cmr_df["cmRate"].apply(lambda x: categorize_cmrates(x, quartile_1,
                                                                               median, quartile_3))
cmr_df.head()

,cmRate,population,medianIncome,pctPoverty,avgHouse,medianAge,medianAgeM,medianAgeF,pctHS,pctBach,...,pctAsian,pctOther,rmRace,pctMarried,pctMarriedHouse,birthRate,region,state,county,cmClass
0,191.2,25859,38013,18.6,2.28,45.4,44.6,46.2,36.4,7.9,...,0.338357,0.065365,white,62.1,57.173258,4.687790,Southeast,Alabama,Cherokee County,High
1,187.9,82005,39922,17.2,2.55,40.5,39.4,41.9,34.7,9.1,...,0.557031,0.737356,white,57.6,54.850257,3.913309,Southeast,Alabama,Cullman County,High
2,217.4,41131,26602,35.2,2.53,38.7,36.2,40.0,34.6,8.2,...,0.341605,0.161313,black,33.4,30.987632,8.433257,Southeast,Alabama,Dallas County,High
3,193.8,81468,54298,14.4,2.65,37.9,36.7,39.1,35.4,13.4,...,0.516326,0.678529,white,51.0,54.326202,5.800147,Southeast,Alabama,Elmore County,High
4,205.8,16759,35664,20.6,2.36,43.2,41.3,46.1,38.8,8.4,...,0.390625,0.000000,white,53.6,51.499218,5.730897,Southeast,Alabama,Fayette County,High


In [108]:
# df = cmr_df.copy()

# # --- RATIOS ---
# df["poverty_to_income"] = df["pctPoverty"] / (df["medianIncome"] + 1)  # avoid division by 0

# # --- INTERACTIONS ---
# df["income_x_education"] = df["medianIncome"] * df["pctBach"]      # higher income + more education
# df["poverty_x_unemployment"] = df["pctPoverty"] * df["pctUnemployed"]  # socioeconomic stress

# # --- DOMAIN-INFORMED FEATURES ---
# df["house_income_ratio"] = df["avgHouse"] / (df["medianIncome"] + 1)
# df["public_private_hc_ratio"] = df["pctPublicHC"] / (df["pctPrivateHC"] + 1e-5)

# Optional: scale these later in preprocessing pipeline


In [135]:
# log_transform_cols = ["population", "medianIncome", "pctPoverty"]
X = cmr_df.drop(columns=["cmRate", "cmClass", "medianAgeM",
                         "medianAgeF", "pctMarriedHouse", "rmRace"])
y = cmr_df["cmClass"]

cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(include="number").columns

In [136]:
# Create a pipeline for numeric features
# numeric_pipeline = Pipeline([
#     # Log-transform selected features
#     ("log", ColumnTransformer([
#         ("log_features", FunctionTransformer(np.log1p, validate=True), log_transform_cols)
#     ], remainder="passthrough")),  # keep other numeric features unchanged
#     ("scaler", StandardScaler())
# ])
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [137]:
# Logistic Regression
log_clf = Pipeline([
    ("prep", preprocess),
    ("model", LogisticRegression(
        multi_class="multinomial",
        max_iter=2000
    ))
])
log_clf.fit(X_train, y_train)

# Random Forest
rf_clf = Pipeline([
    ("prep", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        class_weight="balanced"
    ))
])

rf_clf.fit(X_train, y_train)

# Gradient Boost
gb_clf = Pipeline([
    ("prep", preprocess),
    ("model", GradientBoostingClassifier(
        random_state=42
    ))
])

gb_clf.fit(X_train, y_train)

# SVM
svm_clf = Pipeline([
    ("prep", preprocess),
    ("model", SVC(
        kernel="rbf",
        probability=True,
        class_weight="balanced"
    ))
])

svm_clf.fit(X_train, y_train)

# kNN
knn_clf = Pipeline([
    ("prep", preprocess),
    ("model", KNeighborsClassifier(
        n_neighbors=7,        # You can tune this
        weights="distance"    # distance weighting often improves performance
    ))
])

knn_clf.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['population', 'medianIncome', 'pctPoverty', 'avgHouse', 'medianAge',
       'pctHS', 'pctBach', 'pctEmployed', 'pctUnemployed', 'pctPrivateHC',
       'pctEmployerHC', 'pctPublicHC', 'pctWhite', 'pctBlack', 'pctAsian',
       'pctOther', 'pctMarried', 'birthRate'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['region', 'state', 'county'], dtype='object'))])),
                ('model',
                 KNeighborsClassifier(n_neighbors=7, weights='distance'))])

In [138]:
models = {
    "Logistic Regression": log_clf,
    "Random Forest": rf_clf,
    "Gradient Boosting": gb_clf,
    "SVM (RBF)": svm_clf,
    "KNN": knn_clf
}

for name, model in models.items():
    preds = model.predict(X_test)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Macro F1:", f1_score(y_test, preds, average="macro"))
    print("Weighted F1:", f1_score(y_test, preds, average="weighted"))
    print(classification_report(y_test, preds))


=== Logistic Regression ===
Accuracy: 0.5833333333333334
Macro F1: 0.5664539400665927
Weighted F1: 0.5784275188062646
              precision    recall  f1-score   support

           0       0.55      0.41      0.47        29
           1       0.59      0.66      0.62        53
           2       0.59      0.62      0.60        26

    accuracy                           0.58       108
   macro avg       0.58      0.56      0.57       108
weighted avg       0.58      0.58      0.58       108


=== Random Forest ===
Accuracy: 0.6481481481481481
Macro F1: 0.6149726775956285
Weighted F1: 0.6342653309046752
              precision    recall  f1-score   support

           0       0.73      0.38      0.50        29
           1       0.62      0.81      0.70        53
           2       0.67      0.62      0.64        26

    accuracy                           0.65       108
   macro avg       0.67      0.60      0.61       108
weighted avg       0.66      0.65      0.63       108


=== G

In [139]:
# 1. Preprocess X_train and X_test
X_train_proc = preprocess.fit_transform(X_train)
X_test_proc = preprocess.transform(X_test)

# 2. Apply SMOTE on preprocessed numeric array
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_proc, y_train)

# 3. Train Random Forest on oversampled data
rf = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    class_weight="balanced"
)
rf.fit(X_train_res, y_train_res)

# 4. Predict
preds = rf.predict(X_test_proc)
print(f"\n=== RF CLF ===")
print("Accuracy:", accuracy_score(y_test, preds))
print("Macro F1:", f1_score(y_test, preds, average="macro"))
print("Weighted F1:", f1_score(y_test, preds, average="weighted"))
print(classification_report(y_test, preds))


=== RF CLF ===
Accuracy: 0.6296296296296297
Macro F1: 0.6209025439227112
Weighted F1: 0.6280066579500913
              precision    recall  f1-score   support

           0       0.65      0.52      0.58        29
           1       0.65      0.66      0.65        53
           2       0.58      0.69      0.63        26

    accuracy                           0.63       108
   macro avg       0.63      0.62      0.62       108
weighted avg       0.63      0.63      0.63       108



In [114]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Define SVM model (probability=True if you want soft voting later)
svm_model = SVC(probability=True, random_state=42, class_weight="balanced")

# Full pipeline
svm_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("svm", svm_model)
])

# Grid of hyperparameters
param_grid = {
    "svm__C": [0.1, 1, 10],
    "svm__kernel": ["linear", "rbf"],
    "svm__gamma": ["scale", "auto"]
}

# Grid search
grid_svm = GridSearchCV(
    estimator=svm_pipeline,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring="f1_macro",  # since it's multi-class
    n_jobs=-1
)

# Fit
grid_svm.fit(X_train, y_train)

# Best parameters and performance
print("Best Parameters:", grid_svm.best_params_)
y_pred = grid_svm.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro F1:", f1_score(y_test, y_pred, average="macro"))


Best Parameters: {'svm__C': 10, 'svm__gamma': 'auto', 'svm__kernel': 'rbf'}
Accuracy: 0.5092592592592593
Macro F1: 0.4770433027011974


In [115]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Wrap preprocess + RandomForest into a single pipeline
rf_pipeline = Pipeline([
    ("prep", preprocess),
    ("model", RandomForestClassifier(random_state=42, n_estimators=400, class_weight="balanced"))
])

# Define hyperparameter grid to search
param_grid = {
    "model__max_depth": [10, 20, 60, 100, None],
    # "model__min_samples_split": [2, 5],
    # "model__min_samples_leaf": [1, 2],
    "model__max_features": ["sqrt", "log2", None]
}

# GridSearchCV with 5-fold CV, optimizing macro-F1
grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    verbose=2
)

# Fit the grid search
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 15 candidates, totalling 75 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['population', 'medianIncome', 'pctPoverty', 'avgHouse', 'medianAge',
       'pctHS', 'pctBach', 'pctEmployed', 'pctUnemployed', 'pctPrivateHC',
       'pctEmployerHC', 'pctPublicHC', 'pctWhite', 'pctBlack', 'pctAsian',
       'pctOther', 'pctMarried', 'birthRate'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         Index(['region', 'state', 'county'], dtype='object'))])),
                                       ('model',
                                        RandomForestClassifier(class_weight='balanced',
                                                               n_estimators=400,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [10, 20, 60, 100, None],
                         'model__max_features': ['sqrt', 'log2', None]},
             scoring='f1_macro', verbose=2)

In [116]:
best_rf = grid_search.best_estimator_
best_preds = best_rf.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, classification_report

print("=== Best Random Forest ===")
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, best_preds))
print("Macro F1:", f1_score(y_test, best_preds, average="macro"))
print("Weighted F1:", f1_score(y_test, best_preds, average="weighted"))
print("\nClassification Report:")
print(classification_report(y_test, best_preds))


=== Best Random Forest ===
Best Parameters: {'model__max_depth': 10, 'model__max_features': 'sqrt'}
Accuracy: 0.6296296296296297
Macro F1: 0.5905526917238043
Weighted F1: 0.6268767829437965

Classification Report:
              precision    recall  f1-score   support

        High       0.75      0.72      0.74        54
         Low       0.65      0.74      0.69        27
      Medium       0.36      0.33      0.35        27

    accuracy                           0.63       108
   macro avg       0.59      0.60      0.59       108
weighted avg       0.63      0.63      0.63       108



In [117]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score

# === Select feature types ===
cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(include="number").columns

# === PCA on numeric features only ===
preprocess_pca = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("scaler", StandardScaler()),
            ("pca", PCA(n_components=10))   # you can adjust components
        ]), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

# === Random Forest model ===
rf_pca = Pipeline(steps=[
    ("preprocess", preprocess_pca),
    ("model", RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced"  # helps macro F1 and minority classes
    ))
])

# Train
rf_pca.fit(X_train, y_train)

# Predict
y_pred = rf_pca.predict(X_test)

# Evaluate
print("=== Random Forest + PCA Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Macro F1:", f1_score(y_test, y_pred, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred, average="weighted"))


=== Random Forest + PCA Results ===
Accuracy: 0.6296296296296297
Macro F1: 0.5413238855861807
Weighted F1: 0.5986158650093076
